In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import library as lib
import seaborn as sns
import numpy as np 
from scipy import stats

In [2]:
image_filepath = "../images/plots/attributes"

In [3]:
#import data without duplicates and without scaled
df = lib.import_music_df_with_model(with_scaling = False, remove_dups = True)
df.head()

,Track,Artist,Placement,Year,Month,Day,danceability,energy,key,loudness,speechiness,acousticness,liveness,valence,tempo,Cluster
0,El Paso,Marty Robbins,1,1960,1,4,0.654,0.452,2.0,-9.709,0.0300,0.835,0.160,0.691,106.662,1
4,Why,Frankie Avalon,2,1960,1,4,0.510,0.349,5.0,-8.677,0.0264,0.762,0.130,0.586,94.267,1
7,The Big Hurt,Miss Toni Fisher,3,1960,1,4,0.624,0.557,0.0,-8.309,0.0299,0.783,0.173,0.676,123.919,0
14,Heartaches By The Number,Guy Mitchell,6,1960,1,4,0.584,0.589,5.0,-11.203,0.0718,0.463,0.069,0.842,172.484,2
16,It's Time To Cry,Paul Anka,7,1960,1,4,0.311,0.314,3.0,-12.810,0.0325,0.731,0.335,0.565,109.983,0


In [4]:
df['Cluster'].value_counts().plot(kind='bar')
# we can see cluster 0 is the majority followed by cluster 1 and cluster 2
plt.title('Counts of Songs by Cluster')
plt.ylabel('Counts')
plt.xlabel('Cluster')
sns.set_theme()
plt.savefig(f'{image_filepath}/total_songs_cluster.png')

SyntaxError: invalid syntax (<ipython-input-4-01d795011629>, line 6)

In [ ]:
df = lib.import_music_df_with_model()
df.head() # the correlation coefficient between each feature
attr_corr= df.iloc[:,6:].corr()
attr_corr

In [ ]:
# visuallize the correlation for all features
import numpy as np
# modified by https://seaborn.pydata.org/examples/many_pairwise_correlations.html
# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(attr_corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(attr_corr, mask=mask, cmap=cmap, vmin=-.5,vmax=.5, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

sns.set_theme()

In [ ]:
# create columns to store the labels,which is done by deviding the values for each attribute into 5 groups
df_label = df.iloc[:,6:]
df_label['count'] = 1
for i in (df_label.columns[:-2]):
    df_label[f'{i}_label'] = list(pd.cut(df_label[i],bins=5))
df_label = df_label.iloc[:,9:]
df_label.head()

In [ ]:
#calculate the independance between each features and cluster with Chi Square Test
attribute_list = []
sig_list = []
p_value_list = []


for i in (df.columns[6:-1]):
    attri_raw = df_label[[f'{i}_label','count','Cluster']]

        
    cumulated_feature_by_cluster = attri_raw.groupby([f'{i}_label','Cluster'])['count'].sum()


    chi_df =  cumulated_feature_by_cluster.unstack().fillna(0)
    chi_df_np = chi_df.to_numpy()
    
    
    attribute_list.append(i)
    
    chi_squared_stat, p_value, df_chi, expected_crosstab = stats.chi2_contingency(chi_df_np)
    if p_value < 0.05:
        sig_list.append('O')
    else:
        sig_list.append(' ')
    p_value_list.append(p_value)

sig_df = pd.DataFrame({'attribute':attribute_list,'significant':sig_list,'p_value':p_value_list})
sig_df = sig_df.sort_values('p_value').reset_index(drop=True)
sig_df

In [ ]:
label_list=sig_df['attribute'][:-1]
def dist_plot_cluster_percent(label_list=label_list):
    for i in label_list:
        fig, axes = plt.subplots(1,3,sharex=True,sharey=True,figsize=(15,5))
        plt.subplots_adjust(wspace = 0.5,hspace=0.5)
        fig.suptitle(f'Distribution Plot for [{i}]',y=1.05)
        attri_raw=df_label[[f'{i}_label','count','Cluster']]
        x_axis=[]
        for l in list(attri_raw[f'{i}_label'].sort_values().unique()):
            x_axis.append(str(l))        
       
        if i == 'tempo':
            for c in list(range(3)):
                pd_raw=attri_raw[attri_raw['Cluster']==c]
                pd_series=pd_raw[f'{i}_label']
                mode=pd_series.value_counts().index[0]
                color=['lightblue', 'lightblue', 'lightblue', 'lightblue', 'lightblue']
                color[x_axis.index(str(mode))]='darkred'
                y_axises={0:[0,0,31651,989,0],1:[11,14601,11682,0,0],2:[0,0,0,10160,811]}
                y_axis = [i/sum(y_axises[c])*100 for i in y_axises[c]]
                axes[c].bar(x=x_axis,height=y_axis,color=color)
                axes[c].set_title(f'The mode of {i} for \n[Cluster {c}] is {mode}')
                axes[c].set_xticklabels(x_axis, rotation=90)
                axes[0].set_ylabel('Percentage')
            plt.savefig(f'{image_filepath}/{i}_percent'.lower(),bbox_inches='tight')
                
        else:
            for c in list(range(3)):
                pd_raw=attri_raw[attri_raw['Cluster']==c]
                pd_series=pd_raw[f'{i}_label']
                mode=pd_series.value_counts().index[0]
                color=['lightblue', 'lightblue', 'lightblue', 'lightblue', 'lightblue']
                color[x_axis.index(str(mode))]='darkred'
                y_axis=pd_raw.groupby(f'{i}_label')['count'].sum()
                total=sum(y_axis)
                y_axis=[i/total*100 for i in y_axis]
                axes[c].bar(x=x_axis,height=y_axis,color=color)
                axes[c].set_title(f'The mode of {i} for \n[Cluster {c}] is {mode}')
                axes[c].set_xticklabels(x_axis, rotation=90)
                axes[0].set_ylabel('Percentage')
            plt.savefig(f'{image_filepath}/{i}_percent'.lower(),bbox_inches='tight')
        
        sns.set_theme()
        plt.show()
    

In [ ]:
dist_plot_cluster_percent()

In [ ]:
X=df.iloc[:,6:-1]
y=df.iloc[:,-1]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(max_depth=3,n_estimators=100,random_state=1)
clf.fit(X_train, y_train)
clf.feature_importances_

In [ ]:
predictions = clf.predict(X_test)
errors = abs(predictions - y_test)
total_error_counts=[1 if i!=0 else 0 for i in errors]
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

In [ ]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (sum(total_error_counts) / len(y_test))
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')
pd.DataFrame({'feature':X_train.columns,'importance':clf.feature_importances_}).\
    sort_values('importance',ascending=False)

In [ ]:
pd.DataFrame({'feature':X_train.columns,'importance':clf.feature_importances_}).\
    sort_values('importance',ascending=False)
